In [1]:
#Q1
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
class MySequential_list(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.modules_list = []
        for idx, module in enumerate(args):
            self.modules_list.append(module)

    def forward(self, x):
        for block in self.modules_list:
            x = block(x)
        return x


In [3]:
net_Q1 = MySequential_list([nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)])
X = torch.rand(2, 20)
prednet_test = net_Q1(X)  # TypeError: 'list' object is not callable
print(prednet_test)
print(net_Q1)

TypeError: 'list' object is not callable

In [5]:
#Q2
class ParallelBlock(nn.Module):
    def __init__(self, net1, net2):
        super(ParallelBlock, self).__init__()
        self.net1 = net1
        self.net2 = net2

    def forward(self, x):
        output1 = self.net1(x)
        output2 = self.net2(x)
        return torch.cat((output1, output2), dim=1)
# 创建两个子网络
net1 = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

net2 = nn.Sequential(
    nn.Linear(20, 128),
    nn.ReLU(),
    nn.Linear(128, 5)
)

In [7]:
# 创建平行块
parallel_block = ParallelBlock(net1, net2)

# 测试前向传播
x = torch.randn(2, 20)  # 创建输入张量
output = parallel_block(x)  # 前向传播
print(output)  
print(output.shape)# 输出形状为 (2, 15)，因为 net1 输出 10 维，net2 输出 5 维，串联后为 15 维

tensor([[ 0.0449, -0.3026,  0.1369,  0.3758, -0.0751,  0.2671, -0.1622,  0.2401,
         -0.1306, -0.0428, -0.0774, -0.1692,  0.6624, -0.2798, -0.1921],
        [ 0.1171, -0.1901,  0.1484,  0.3833, -0.3507, -0.3173,  0.0095, -0.2878,
          0.2175,  0.0653, -0.2872,  0.0158, -0.0486, -0.1562, -0.2522]],
       grad_fn=<CatBackward0>)
torch.Size([2, 15])


In [ ]:
#Q3有点问题这里的features的特征只能先后一样，目前还没有想到好的办法解决。

In [22]:
class Basic_block(nn.Module):
    def __init__(self):
        super(Basic_block, self).__init__()
        self.linear = nn.Linear(20, 20)  # 输入维度为20，输出维度为50
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.linear(x))

def create_more(block_class, num_blocks, *args, **kwargs):
    blocks = []
    for _ in range(num_blocks):
        blocks.append(block_class(*args, **kwargs))
    return nn.Sequential(*blocks)

# 创建由5个 Basic_block 实例组成的更大网络
larger_network = create_more(Basic_block, 5)

# 测试前向传播
x = torch.randn(2, 20)  # 创建输入张量，维度为2x20
output = larger_network(x)  # 前向传播
print(output.shape)  # 输出张量的形状


torch.Size([2, 20])
